In [2]:
import pandas as pd
import torch
# import seaborn as sns
from dl_biology.cafa6.embedding import EmbeddingGenerator
%load_ext autoreload
%autoreload 2

/Users/tungvuduc/opt/anaconda3/envs/dlbio_arm64/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# filepaths to fasta
path_to_fasta_train = "/Users/tungvuduc/Desktop/PhD/projects/repos/Deep-learning-for-biology/data/kaggle/cafa-6-protein-function-prediction/Train/train_sequences.fasta"
path_to_fasta_test = "/Users/tungvuduc/Desktop/PhD/projects/repos/Deep-learning-for-biology/data/kaggle/cafa-6-protein-function-prediction/Test/testsuperset.fasta"

# filepath to GO terms
path_to_go_term_train = "/Users/tungvuduc/Desktop/PhD/projects/repos/Deep-learning-for-biology/data/kaggle/cafa-6-protein-function-prediction/Train/train_terms.tsv"

In [4]:
# let's read the data
train_terms = pd.read_csv(path_to_go_term_train, delimiter="\t")

In [7]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

embed_gen = EmbeddingGenerator(device=device, model_checkpoint="facebook/esm2_t33_650M_UR50D")

output_dir = "/Users/tungvuduc/Desktop/PhD/projects/repos/Deep-learning-for-biology/data/kaggle/cafa-6-protein-function-prediction/train_hidden_states"
fasta_file = "/Users/tungvuduc/Desktop/PhD/projects/repos/Deep-learning-for-biology/data/kaggle/cafa-6-protein-function-prediction/Train/train_sequences.fasta"
embed_gen.generate_embedding(
    output_dir=output_dir,
    fasta_file=fasta_file,
    batch_size=64,
    max_length=1024
)

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
16it [07:23, 27.71s/it]


In [23]:
from pathlib import Path
import h5py

h5_path = "/Users/tungvuduc/Desktop/PhD/projects/repos/Deep-learning-for-biology/data/kaggle/cafa-6-protein-function-prediction/train_hidden_states/train_embeddings.h5"

h5_path = Path(h5_path)

# Get all sequence IDs from the HDF5 file
with h5py.File(h5_path, 'r') as h5f:
    seq_ids = list(h5f.keys())

with h5py.File(h5_path, 'r') as h5f:
    grp = h5f[seq_ids[1]]
    embedding = torch.from_numpy(grp['embedding'][:])

train_terms_trunc = train_terms[train_terms["EntryID"].isin(seq_ids)]
cross_tab_train_term = pd.crosstab(train_terms_trunc['EntryID'], train_terms_trunc['term']).values
embedding.shape


torch.Size([1024, 1280])

In [24]:
cross_tab_train_term

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(1000, 4165))

In [12]:
cross_tab_train_term


term,GO:0000012,GO:0000016,GO:0000017,GO:0000028,GO:0000045,GO:0000049,GO:0000050,GO:0000062,GO:0000070,GO:0000076,...,GO:2001171,GO:2001190,GO:2001205,GO:2001234,GO:2001235,GO:2001237,GO:2001238,GO:2001240,GO:2001241,GO:2001244
EntryID,,,,,,,,,,,,,,,,,,,,,
A0A0C5B5G6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0JNW5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0JP26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0PK11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1A4S6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P59827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P59998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P60022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
